**Lab M1.06 - API Calling with JSON**

Dina Bosma-Buczynska


**Step 1: Setting Up Pydantic**

In [2]:
"""
API with JSON Validation using Pydantic - Complete Solution
Validate JSON input using Pydantic before processing
"""
 
from pydantic import BaseModel, Field, validator
from typing import Optional
import json
 
print("="*50)
print("PYDANTIC BASICS")
print("="*50)
 
class SimpleProduct(BaseModel):
    """A simple product model for validation."""
    name: str
    price: float
    quantity: int = 1  # Default value
    
    @validator('price')
    def price_must_be_positive(cls, v):
        """Validate that price is positive."""
        if v <= 0:
            raise ValueError('Price must be positive')
        return v
    
    @validator('quantity')
    def quantity_must_be_positive(cls, v):
        """Validate that quantity is positive."""
        if v <= 0:
            raise ValueError('Quantity must be positive')
        return v
 
# Test validation
print("\n1. Valid data:")
try:
    product1 = SimpleProduct(name="Widget", price=10.99, quantity=5)
    print(f"  ✓ Valid: {product1.name} - ${product1.price}")
except Exception as e:
    print(f"  ✗ Error: {e}")
 
print("\n2. Invalid data (negative price):")
try:
    product2 = SimpleProduct(name="Widget", price=-10.99)
except Exception as e:
    print(f"  ✗ Validation error (expected): {e}")
 
print("\n✓ Pydantic basics working!")

PYDANTIC BASICS

1. Valid data:
  ✓ Valid: Widget - $10.99

2. Invalid data (negative price):
  ✗ Validation error (expected): 1 validation error for SimpleProduct
price
  Value error, Price must be positive [type=value_error, input_value=-10.99, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

✓ Pydantic basics working!


/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_4628/3349469637.py:20: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('price')
/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_4628/3349469637.py:27: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('quantity')



**Step 2: Creating Product Data Models**

Client sends JSON →  Validator checks it  →  If valid, process with ChatGPT →   If invalid, reject with clear error
                     

In [4]:
# ==============================================
# STEP 2: Step 2: Creating Product Data Models
# ==============================================

print("\n" + "="*50)
print("STEP 2: COMPLETE PRODUCT MODEL")
print("="*50)

class ProductListing(BaseModel):
    """
    Complete product model for e-commerce listings.
    This validates ALL the data clients send us!
    """
    
    # =================
    # REQUIRED FIELDS 
    # ==================
    name: str = Field(
        ...,  # ... means "REQUIRED - no default value"
        min_length=3,
        max_length=100,
        description="Product name (3-100 characters)"
    )
    
    price: float = Field(
        ...,  # REQUIRED
        gt=0,  # gt = greater than (must be positive)
        le=1000000,  # le = less than or equal to (max $1 million)
        description="Product price ($0.01 - $1,000,000)"
    )
    
    category: str = Field(
        ...,  # REQUIRED
        min_length=2,
        description="Product category"
    )
    
    description: str = Field(
        ...,  # REQUIRED
        min_length=10,
        max_length=500,
        description="Product description (10-500 characters)"
    )
    
    # ================
    # OPTIONAL FIELDS 
    # =================
    brand: Optional[str] = Field(
        None,  # None = optional, defaults to None
        max_length=50,
        description="Brand name (optional)"
    )
    
    features: Optional[list[str]] = Field(
        default_factory=list,  # If missing, create empty list
        description="List of product features (optional)"
    )
    
    image_url: Optional[str] = Field(
        None,
        description="Product image URL (optional)"
    )
    
    in_stock: bool = Field(
        True,  # Default to True
        description="Whether product is in stock"
    )
    
    # ===================
    # CUSTOM VALIDATORS 
    # ===================
    
    @validator('name')
    def name_must_not_be_empty(cls, v):
        """
        Ensure name isn't just spaces.
        Example: "   " should be rejected!
        """
        if not v or v.strip() == "":
            raise ValueError('Name cannot be empty or just spaces')
        # Clean up: remove leading/trailing spaces
        return v.strip()
    
    @validator('category')
    def category_must_be_valid(cls, v):
        """
        Check if category is in our allowed list.
        Think of this like a restaurant menu - only certain categories allowed!
        """
        valid_categories = [
            "Electronics",
            "Clothing", 
            "Home & Garden",
            "Sports",
            "Books",
            "Toys",
            "Food"
        ]
        
        if v not in valid_categories:
            raise ValueError(
                f'Category must be one of: {", ".join(valid_categories)}. '
                f'You provided: "{v}"'
            )
        return v
    
    @validator('price')
    def price_validation(cls, v):
        """
        Extra price validation beyond just "must be positive".
        - Must be at least 1 cent
        - Can't exceed $1 million
        - Round to 2 decimal places (no $19.999999)
        """
        if v < 0.01:
            raise ValueError('Price must be at least $0.01')
        
        if v > 1000000:
            raise ValueError('Price cannot exceed $1,000,000')
        
        # Round to 2 decimals: 19.999 becomes 19.99
        return round(v, 2)
    
    @validator('description')
    def description_must_be_meaningful(cls, v):
        """
        Description should be actual text, not just spaces or dots.
        """
        cleaned = v.strip()
        if len(cleaned) < 10:
            raise ValueError('Description must be at least 10 characters (excluding spaces)')
        return cleaned
    
    @validator('features')
    def features_must_be_valid(cls, v):
        """
        Validate features list if provided.
        - Maximum 10 features (no spam!)
        - Remove empty strings
        - Remove duplicates
        """
        if v is not None:
            # Remove empty features
            v = [f.strip() for f in v if f.strip()]
            
            # Check max limit
            if len(v) > 10:
                raise ValueError('Maximum 10 features allowed')
            
            # Remove duplicates while preserving order
            seen = set()
            v = [x for x in v if not (x in seen or seen.add(x))]
        
        return v
    
    @validator('image_url')
    def image_url_must_be_valid(cls, v):
        """
        Basic URL validation - must start with http:// or https://
        """
        if v is not None and v.strip():
            if not v.startswith(('http://', 'https://')):
                raise ValueError('Image URL must start with http:// or https://')
        return v

print("\n✓ ProductListing model created!")
print("\nModel includes:")
print("  • 4 required fields: name, price, category, description")
print("  • 4 optional fields: brand, features, image_url, in_stock")
print("  • 6 custom validators for business logic")


STEP 2: COMPLETE PRODUCT MODEL

✓ ProductListing model created!

Model includes:
  • 4 required fields: name, price, category, description
  • 4 optional fields: brand, features, image_url, in_stock
  • 6 custom validators for business logic


/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_4628/740203329.py:73: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('name')
/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_4628/740203329.py:84: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('category')
/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_4628/740203329.py:107: PydanticDeprecatedSince20: Pydantic V1 style `@validat

**Checkpoint:**

Verify that:

- Valid data passes validation
- Invalid data raises appropriate errors
- Required fields are enforced
- Custom validators work correctly



In [6]:
print("\n" + "="*50)
print("TESTING PRODUCT MODEL")
print("="*50)

# TEST 1: Valid product with ALL fields
print("\n1. Creating valid product with all fields...")
print("-"*50)
try:
    product_full = ProductListing(
        name="Wireless Bluetooth Headphones Pro",
        price=149.99,
        category="Electronics",
        description="Premium wireless headphones with active noise cancellation, 30-hour battery life, and superior sound quality. Perfect for music lovers and frequent travelers.",
        brand="AudioTech",
        features=["Noise Cancellation", "30hr Battery", "Bluetooth 5.0", "Foldable"],
        image_url="https://example.com/headphones.jpg",
        in_stock=True
    )
    print("✓ SUCCESS! Product created:")
    print(f"  Name: {product_full.name}")
    print(f"  Price: ${product_full.price}")
    print(f"  Category: {product_full.category}")
    print(f"  Brand: {product_full.brand}")
    print(f"  Features: {len(product_full.features)} items")
    print(f"  In Stock: {product_full.in_stock}")
except Exception as e:
    print(f"✗ FAILED: {e}")


TESTING PRODUCT MODEL

1. Creating valid product with all fields...
--------------------------------------------------
✓ SUCCESS! Product created:
  Name: Wireless Bluetooth Headphones Pro
  Price: $149.99
  Category: Electronics
  Brand: AudioTech
  Features: 4 items
  In Stock: True


In [7]:
# TEST 2: Valid product with ONLY required fields
print("\n2. Creating valid product with only required fields...")
print("-"*50)
try:
    product_minimal = ProductListing(
        name="Simple Toy Widget",
        price=9.99,
        category="Toys",
        description="A colorful educational toy for children aged 3 and up. Promotes creativity and motor skills."
    )
    print("✓ SUCCESS! Minimal product created:")
    print(f"  Name: {product_minimal.name}")
    print(f"  Price: ${product_minimal.price}")
    print(f"  Brand: {product_minimal.brand} (None = not provided)")
    print(f"  Features: {product_minimal.features} (empty list = not provided)")
    print(f"  In Stock: {product_minimal.in_stock} (True = default)")
except Exception as e:
    print(f"✗ FAILED: {e}")


2. Creating valid product with only required fields...
--------------------------------------------------
✓ SUCCESS! Minimal product created:
  Name: Simple Toy Widget
  Price: $9.99
  Brand: None (None = not provided)
  Features: [] (empty list = not provided)
  In Stock: True (True = default)


In [12]:
# TEST 3: Invalid - Name too short
print("\n3. Testing invalid name (too short)...")
print("-"*50)
try:
    product_bad1 = ProductListing(
        name="AB",  # Only 2 characters.
        price=10.00,
        category="Toys",
        description="This should fail because name is too short."
    )
    print("✗ UNEXPECTED: Should have failed but didn't!")
except Exception as e:
    print(f"✓ CAUGHT ERROR (expected):")
    print(f"  {e}")


3. Testing invalid name (too short)...
--------------------------------------------------
✓ CAUGHT ERROR (expected):
  1 validation error for ProductListing
name
  String should have at least 3 characters [type=string_too_short, input_value='AB', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short


In [11]:
# TEST 4: Invalid - Wrong category
print("\n4. Testing invalid category...")
print("-"*50)
try:
    product_bad2 = ProductListing(
        name="Mystery Item",
        price=10.00,
        category="Magical Unicorns",  # Not in the list!
        description="This should fail because category is invalid."
    )
    print("✗ UNEXPECTED: Should have failed but didn't!")
except Exception as e:
    print(f"✓ CAUGHT ERROR (expected):")
    print(f"  {e}")


4. Testing invalid category...
--------------------------------------------------
✓ CAUGHT ERROR (expected):
  1 validation error for ProductListing
category
  Value error, Category must be one of: Electronics, Clothing, Home & Garden, Sports, Books, Toys, Food. You provided: "Magical Unicorns" [type=value_error, input_value='Magical Unicorns', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [13]:
# TEST 5: Invalid - Negative price
print("\n5. Testing invalid price (negative)...")
print("-"*50)
try:
    product_bad3 = ProductListing(
        name="Free Giveaway Item",
        price=-5.00,  # Negative!
        category="Toys",
        description="This should fail because price is negative."
    )
    print("✗ UNEXPECTED: Should have failed but didn't!")
except Exception as e:
    print(f"✓ CAUGHT ERROR (expected):")
    print(f"  {e}")


5. Testing invalid price (negative)...
--------------------------------------------------
✓ CAUGHT ERROR (expected):
  1 validation error for ProductListing
price
  Input should be greater than 0 [type=greater_than, input_value=-5.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


In [15]:
# TEST 6: Price rounding
print("\n6. Testing price rounding...")
print("-"*50)
try:
    product_round = ProductListing(
        name="Precisely Priced Widget",
        price=19.995,  # Should round up to 20.00
        category="Toys",
        description="Testing automatic price rounding to 2 decimal places."
    )
    print(f"✓ SUCCESS! Price automatically rounded:")
    print(f"  Input: 19.995")
    print(f"  Output: ${product_round.price:.2f}")
    
    # Test another rounding case
    product_round2 = ProductListing(
        name="Another Priced Widget",
        price=19.994,  # Should round up to 19.99
        category="Toys", 
        description="Testing rounding down to nearest cent."
    )
    print(f"\n  Second test:")
    print(f"  Input: 19.994")
    print(f"  Output: ${product_round2.price:.2f}")
    print(f"\n  (Validator rounded both correctly!)")
except Exception as e:
    print(f"✗ FAILED: {e}")


6. Testing price rounding...
--------------------------------------------------
✓ SUCCESS! Price automatically rounded:
  Input: 19.995
  Output: $20.00

  Second test:
  Input: 19.994
  Output: $19.99

  (Validator rounded both correctly!)


In [16]:
# TEST 7: Features cleaning
print("\n7. Testing features list cleaning...")
print("-"*50)
try:
    product_features = ProductListing(
        name="Feature-Rich Gadget",
        price=79.99,
        category="Electronics",
        description="Testing how the validator cleans up the features list automatically.",
        features=["WiFi", "", "Bluetooth", "WiFi", "USB-C", "  "]  # Has empties and duplicates!
    )
    print(f"✓ SUCCESS! Features automatically cleaned:")
    print(f"  Input: ['WiFi', '', 'Bluetooth', 'WiFi', 'USB-C', '  ']")
    print(f"  Output: {product_features.features}")
    print(f"  (Removed empties and duplicates!)")
except Exception as e:
    print(f"✗ FAILED: {e}")


7. Testing features list cleaning...
--------------------------------------------------
✓ SUCCESS! Features automatically cleaned:
  Input: ['WiFi', '', 'Bluetooth', 'WiFi', 'USB-C', '  ']
  Output: ['WiFi', 'Bluetooth', 'USB-C']
  (Removed empties and duplicates!)


In [17]:
# TEST 8: Missing required field
print("\n8. Testing missing required field...")
print("-"*50)
try:
    product_missing = ProductListing(
        name="Incomplete Product",
        price=15.00,
        category="Toys"
        # Is missing the description!
    )
    print("✗ UNEXPECTED: Should have failed but didn't!")
except Exception as e:
    print(f"✓ CAUGHT ERROR (expected):")
    print(f"  {e}")


8. Testing missing required field...
--------------------------------------------------
✓ CAUGHT ERROR (expected):
  1 validation error for ProductListing
description
  Field required [type=missing, input_value={'name': 'Incomplete Prod...5.0, 'category': 'Toys'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing


In [18]:
# TEST 9: Name trimming
print("\n9. Testing name trimming...")
print("-"*50)
try:
    product_trim = ProductListing(
        name="   Spaced Out Widget   ",  # Has leading/trailing spaces
        price=12.99,
        category="Toys",
        description="Testing automatic trimming of leading and trailing spaces from name."
    )
    print(f"✓ SUCCESS! Name automatically trimmed:")
    print(f"  Input: '   Spaced Out Widget   '")
    print(f"  Output: '{product_trim.name}'")
    print(f"  (Validator removed the spaces!)")
except Exception as e:
    print(f"✗ FAILED: {e}")


9. Testing name trimming...
--------------------------------------------------
✓ SUCCESS! Name automatically trimmed:
  Input: '   Spaced Out Widget   '
  Output: 'Spaced Out Widget'
  (Validator removed the spaces!)


**Step 3: Validating JSON Input**


In [19]:
import os

print("="*50)
print("STEP 3: CREATING TEST JSON FILES")
print("="*50)

# Create a directory for test data
os.makedirs('test_data', exist_ok=True)
print("\n✓ Created 'test_data' directory")

# FILE 1: Valid product with all fields
valid_product = {
    "name": "Premium Wireless Headphones",
    "price": 149.99,
    "category": "Electronics",
    "description": "High-quality wireless headphones with active noise cancellation, 30-hour battery life, and premium sound quality. Perfect for music lovers and frequent travelers.",
    "brand": "AudioTech Pro",
    "features": [
        "Active Noise Cancellation",
        "30-hour battery life",
        "Bluetooth 5.0",
        "Premium sound quality"
    ],
    "image_url": "https://example.com/headphones.jpg",
    "in_stock": True
}

with open('test_data/valid_product.json', 'w') as f:
    json.dump(valid_product, f, indent=2)

print("✓ Created: test_data/valid_product.json")

STEP 3: CREATING TEST JSON FILES

✓ Created 'test_data' directory
✓ Created: test_data/valid_product.json


In [20]:
# FILE 2: Invalid product - negative price
invalid_price = {
    "name": "Broken Product",
    "price": -25.00,
    "category": "Electronics",
    "description": "This product has an invalid negative price and should be rejected by validation."
}

with open('test_data/invalid_product_price.json', 'w') as f:
    json.dump(invalid_price, f, indent=2)

print("✓ Created: test_data/invalid_product_price.json")

# FILE 3: Invalid product - wrong category
invalid_category = {
    "name": "Mystery Box",
    "price": 50.00,
    "category": "Magical Items",
    "description": "This product has an invalid category that isn't in our allowed list."
}

with open('test_data/invalid_product_category.json', 'w') as f:
    json.dump(invalid_category, f, indent=2)

print("✓ Created: test_data/invalid_product_category.json")

# FILE 4: Invalid product - missing required field
invalid_missing = {
    "name": "Incomplete Product",
    "price": 15.00,
    "category": "Toys"
    # Missing description!
}

with open('test_data/invalid_product_missing.json', 'w') as f:
    json.dump(invalid_missing, f, indent=2)

print("✓ Created: test_data/invalid_product_missing.json")

# FILE 5: Invalid product - name too short
invalid_name = {
    "name": "AB",
    "price": 20.00,
    "category": "Toys",
    "description": "This product has a name that's too short (only 2 characters)."
}

with open('test_data/invalid_product_name.json', 'w') as f:
    json.dump(invalid_name, f, indent=2)

print("✓ Created: test_data/invalid_product_name.json")

print("\n" + "="*50)
print("✓ ALL TEST FILES CREATED!")
print("="*50)
print("\nTotal files created: 5")
print("  • 1 valid product")
print("  • 4 invalid products (for testing error handling)")

✓ Created: test_data/invalid_product_price.json
✓ Created: test_data/invalid_product_category.json
✓ Created: test_data/invalid_product_missing.json
✓ Created: test_data/invalid_product_name.json

✓ ALL TEST FILES CREATED!

Total files created: 5
  • 1 valid product
  • 4 invalid products (for testing error handling)


In [21]:
# View what files are created
print("\n" + "="*50)
print("VERIFYING TEST FILES")
print("="*50)

import os
files = os.listdir('test_data')
print(f"\nFiles in test_data directory: {len(files)}")
for file in sorted(files):
    print(f"  • {file}")

# Look at one file
print("\n" + "-"*50)
print("EXAMPLE: Contents of valid_product.json")
print("-"*50)
with open('test_data/valid_product.json', 'r') as f:
    content = f.read()
    print(content)


VERIFYING TEST FILES

Files in test_data directory: 5
  • invalid_product_category.json
  • invalid_product_missing.json
  • invalid_product_name.json
  • invalid_product_price.json
  • valid_product.json

--------------------------------------------------
EXAMPLE: Contents of valid_product.json
--------------------------------------------------
{
  "name": "Premium Wireless Headphones",
  "price": 149.99,
  "category": "Electronics",
  "description": "High-quality wireless headphones with active noise cancellation, 30-hour battery life, and premium sound quality. Perfect for music lovers and frequent travelers.",
  "brand": "AudioTech Pro",
  "features": [
    "Active Noise Cancellation",
    "30-hour battery life",
    "Bluetooth 5.0",
    "Premium sound quality"
  ],
  "image_url": "https://example.com/headphones.jpg",
  "in_stock": true
}


In [22]:
print("\n" + "="*50)
print("CREATING JSON VALIDATION FUNCTION")
print("="*50)

def validate_product_json(json_file_path):
    """
    Load and validate a product from a JSON file.
    
    This function does 3 things:
    1. Loads the JSON file
    2. Validates it with the Pydantic model
    3. Returns the validated product OR None if invalid
    
    Args:
        json_file_path: Path to the JSON file
        
    Returns:
        ProductListing object if valid, None if invalid
    """
    print(f"\n Loading: {json_file_path}")
    print("-" * 60)
    
    try:
        # STEP 1: Load the JSON file
        with open(json_file_path, 'r') as file:
            json_data = json.load(file)
        
        print(" JSON file loaded successfully")
        
        # Show a preview of what has been loaded
        preview = json.dumps(json_data, indent=2)
        if len(preview) > 300:
            preview = preview[:300] + "\n  ... (truncated)"
        print(f"   Data preview:\n{preview}")
        
        # STEP 2: Validate with Pydantic
        product = ProductListing(**json_data)
        
        print("\n VALIDATION PASSED!")
        print(f"   Product Name: {product.name}")
        print(f"   Price: ${product.price}")
        print(f"   Category: {product.category}")
        print(f"   Description: {product.description[:50]}...")
        if product.brand:
            print(f"   Brand: {product.brand}")
        if product.features:
            print(f"   Features: {len(product.features)} items")
        
        return product
        
    except FileNotFoundError:
        print(f" ERROR: File not found")
        print(f"   Please check that {json_file_path} exists")
        return None
        
    except json.JSONDecodeError as e:
        print(f" ERROR: Invalid JSON format")
        print(f"   The file isn't valid JSON")
        print(f"   Details: {str(e)}")
        return None
        
    except Exception as e:
        print(f" VALIDATION FAILED!")
        print(f"   Reason: {str(e)}")
        
        # Show detailed error information if available
        if hasattr(e, 'errors'):
            print("\n   Detailed validation errors:")
            for error in e.errors():
                # Get the field name that failed
                field = " → ".join(str(loc) for loc in error['loc'])
                # Get the error message
                msg = error['msg']
                # Get the error type
                error_type = error['type']
                
                print(f"     • Field: '{field}'")
                print(f"       Error: {msg}")
                print(f"       Type: {error_type}")
        
        return None

print("\n Validation function created!")
print("\nThe function can handle:")
print("  • Missing files (FileNotFoundError)")
print("  • Invalid JSON syntax (JSONDecodeError)")  
print("  • Validation errors (Pydantic ValidationError)")


CREATING JSON VALIDATION FUNCTION

 Validation function created!

The function can handle:
  • Missing files (FileNotFoundError)
  • Invalid JSON syntax (JSONDecodeError)
  • Validation errors (Pydantic ValidationError)


In [24]:
print("\n" + "="*50)
print("TESTING JSON VALIDATION ON ALL FILES")
print("="*50)

# List of all test files
test_files = [
    "test_data/valid_product.json",
    "test_data/invalid_product_price.json",
    "test_data/invalid_product_category.json",
    "test_data/invalid_product_missing.json",
    "test_data/invalid_product_name.json"
]

# Track results
results = []

# Test each file
for file_path in test_files:
    result = validate_product_json(file_path)
    results.append({
        'file': file_path,
        'valid': result is not None,
        'product': result
    })
    print()  # Empty line between tests

# Summary
print("="*60)
print("VALIDATION SUMMARY")
print("="*60)

for result in results:
    status = " PASSED" if result['valid'] else " FAILED"
    file_name = result['file'].split('/')[-1]
    print(f"{status}: {file_name}")

valid_count = sum(1 for r in results if r['valid'])
total_count = len(results)

print(f"\nTotal: {valid_count}/{total_count} files passed validation")

if valid_count == 1:
    print("\n✓ Expected result: Only 1 file should be valid!")
    print("  (The other 4 should fail with clear error messages)")
else:
    print(f"\n⚠ Warning: Expected 1 valid file, got {valid_count}")




TESTING JSON VALIDATION ON ALL FILES

 Loading: test_data/valid_product.json
------------------------------------------------------------
 JSON file loaded successfully
   Data preview:
{
  "name": "Premium Wireless Headphones",
  "price": 149.99,
  "category": "Electronics",
  "description": "High-quality wireless headphones with active noise cancellation, 30-hour battery life, and premium sound quality. Perfect for music lovers and frequent travelers.",
  "brand": "AudioTech Pro"
  ... (truncated)

 VALIDATION PASSED!
   Product Name: Premium Wireless Headphones
   Price: $149.99
   Category: Electronics
   Description: High-quality wireless headphones with active noise...
   Brand: AudioTech Pro
   Features: 4 items


 Loading: test_data/invalid_product_price.json
------------------------------------------------------------
 JSON file loaded successfully
   Data preview:
{
  "name": "Broken Product",
  "price": -25.0,
  "category": "Electronics",
  "description": "This product has 

**Step 4: Integrating with ChatGPT API**

- Client sends JSON → Product data 
- Validate INPUT → Reject if bad  
- Send to ChatGPT → Generate listing
- Validate OUTPUT → Ensure quality 
- Return result → Happy client! 

In [25]:

!pip install openai

print("✓ OpenAI library installation complete!")

✓ OpenAI library installation complete!


In [31]:
print("="*50)
print("STEP 4: OPENAI API INTEGRATION")
print("="*50)

from openai import OpenAI

import os
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print("\n✓ OpenAI client initialized!")
print("✓ API key configured successfully!")

STEP 4: OPENAI API INTEGRATION

✓ OpenAI client initialized!
✓ API key configured successfully!


In [32]:
print("\n" + "="*50)
print("CREATING OUTPUT VALIDATION MODEL")
print("="*50)

class EnhancedProductListing(BaseModel):
    """
    Model for ChatGPT's enhanced product listing output.
    This ensures ChatGPT returns properly formatted data!
    """
    
    # Original fields (from input)
    name: str
    price: float
    category: str
    
    # Enhanced fields (generated by ChatGPT)
    marketing_description: str = Field(
        ...,
        min_length=50,
        max_length=1000,
        description="Marketing-focused product description"
    )
    
    key_benefits: list[str] = Field(
        ...,
        min_items=3,
        max_items=8,
        description="3-8 key product benefits"
    )
    
    target_audience: str = Field(
        ...,
        min_length=10,
        max_length=200,
        description="Who this product is perfect for"
    )
    
    seo_keywords: list[str] = Field(
        ...,
        min_items=5,
        max_items=15,
        description="SEO keywords for the product"
    )
    
    @validator('key_benefits')
    def benefits_must_be_meaningful(cls, v):
        """Each benefit should be substantial, not just one word."""
        if v:
            for benefit in v:
                if len(benefit.strip()) < 5:
                    raise ValueError('Each benefit must be at least 5 characters')
        return v
    
    @validator('seo_keywords')
    def keywords_must_be_clean(cls, v):
        """Keywords should be clean, lowercase, no duplicates."""
        if v:
            # Convert to lowercase and strip whitespace
            v = [kw.lower().strip() for kw in v]
            # Remove empty strings
            v = [kw for kw in v if kw]
            # Remove duplicates while preserving order
            seen = set()
            v = [x for x in v if not (x in seen or seen.add(x))]
        return v

print("\n✓ EnhancedProductListing model created!")
print("\nThis model validates ChatGPT's output to ensure:")
print("  • Marketing description is 50-1000 characters")
print("  • 3-8 key benefits provided")
print("  • Target audience described (10-200 characters)")
print("  • 5-15 SEO keywords included")
print("  • All fields properly formatted")


CREATING OUTPUT VALIDATION MODEL

✓ EnhancedProductListing model created!

This model validates ChatGPT's output to ensure:
  • Marketing description is 50-1000 characters
  • 3-8 key benefits provided
  • Target audience described (10-200 characters)
  • 5-15 SEO keywords included
  • All fields properly formatted


/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_4628/2179249608.py:24: PydanticDeprecatedSince20: `min_items` is deprecated and will be removed, use `min_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  key_benefits: list[str] = Field(
/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_4628/2179249608.py:24: PydanticDeprecatedSince20: `max_items` is deprecated and will be removed, use `max_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  key_benefits: list[str] = Field(
/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_4628/2179249608.py:38: PydanticDeprecatedSince20: `min_items` is deprecated and will be removed, use `min_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migra

In [46]:
print("\n" + "="*50)
print("CREATING PRODUCT ENHANCEMENT FUNCTION")
print("="*50)

def enhance_product_with_chatgpt(product: ProductListing):
    """
    Takes a validated product and uses ChatGPT to create enhanced listing.
    
    Args:
        product: A validated ProductListing object
        
    Returns:
        EnhancedProductListing object if successful, None if failed
    """
    
    print(f"\n Enhancing product: {product.name}")
    print("-" * 60)
    
    # Build a prompt for ChatGPT
    prompt = f"""You are a professional product marketing specialist. 

Given this product information:
- Name: {product.name}
- Price: ${product.price}
- Category: {product.category}
- Description: {product.description}
{f"- Brand: {product.brand}" if product.brand else ""}
{f"- Features: {', '.join(product.features)}" if product.features else ""}

Create an enhanced product listing with:
1. A compelling marketing description (50-1000 characters)
2. 3-8 key benefits for customers
3. Target audience description
4. 5-15 SEO keywords

Respond ONLY with valid JSON in this exact format:
{{
  "name": "{product.name}",
  "price": {product.price},
  "category": "{product.category}",
  "marketing_description": "your compelling description here",
  "key_benefits": ["benefit 1", "benefit 2", "benefit 3"],
  "target_audience": "who this is perfect for",
  "seo_keywords": ["keyword1", "keyword2", "keyword3", "keyword4", "keyword5"]
}}

IMPORTANT: 
- Return ONLY the JSON object, no other text
- Ensure all fields are included
- Make the marketing description engaging and professional
- Keep benefits concise but meaningful"""

    try:
        # Call ChatGPT API
        print(" Calling ChatGPT API...")
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Using mini for cost efficiency
            messages=[
                {
                    "role": "system",
                    "content": "You are a professional product marketing specialist. Always respond with valid JSON only."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            temperature=0.7,  # Some creativity, but not too crazy
            max_tokens=1000
        )
        
        # Extract the response text
        response_text = response.choices[0].message.content
        print(" ChatGPT response received")
        
        # Parse JSON response
        print(" Parsing JSON response...")
        
        # Sometimes ChatGPT adds markdown code blocks, remove them
        if response_text.strip().startswith("```"):
            # Remove ```json and ``` markers
            response_text = response_text.strip()
            response_text = response_text.replace("```json", "").replace("```", "")
        
        enhanced_data = json.loads(response_text)
        print(" JSON parsed successfully")
        
        # Validate with Pydantic
        print(" Validating output with Pydantic...")
        enhanced_product = EnhancedProductListing(**enhanced_data)
        
        print(" OUTPUT VALIDATION PASSED!")
        print("\n" + "="*60)
        print("ENHANCED PRODUCT LISTING")
        print("="*60)
        print(f"Name: {enhanced_product.name}")
        print(f"Price: ${enhanced_product.price}")
        print(f"Category: {enhanced_product.category}")
        print(f"\nMarketing Description:")
        print(f"  {enhanced_product.marketing_description[:150]}...")
        print(f"\nKey Benefits ({len(enhanced_product.key_benefits)} items):")
        for i, benefit in enumerate(enhanced_product.key_benefits, 1):
            print(f"  {i}. {benefit}")
        print(f"\nTarget Audience:")
        print(f"  {enhanced_product.target_audience}")
        print(f"\nSEO Keywords ({len(enhanced_product.seo_keywords)} items):")
        print(f"  {', '.join(enhanced_product.seo_keywords[:10])}")
        if len(enhanced_product.seo_keywords) > 10:
            print(f"  ... and {len(enhanced_product.seo_keywords) - 10} more")
        
        return enhanced_product
        
    except json.JSONDecodeError as e:
        print(f" ERROR: ChatGPT response is not valid JSON")
        print(f"   Details: {e}")
        print(f"   Response was: {response_text[:200]}...")
        return None
        
    except Exception as e:
        print(f" ERROR: {str(e)}")
        if hasattr(e, 'errors'):
            print("\n   Validation errors:")
            for error in e.errors():
                field = " → ".join(str(loc) for loc in error['loc'])
                print(f"     • Field '{field}': {error['msg']}")
        return None

print("\n✓ Product enhancement function created!")
print("\nThis function:")
print("  1. Takes validated product data")
print("  2. Builds a detailed prompt for ChatGPT")
print("  3. Calls ChatGPT API")
print("  4. Parses JSON response")
print("  5. Validates output with Pydantic")
print("  6. Returns enhanced product listing")


CREATING PRODUCT ENHANCEMENT FUNCTION

✓ Product enhancement function created!

This function:
  1. Takes validated product data
  2. Builds a detailed prompt for ChatGPT
  3. Calls ChatGPT API
  4. Parses JSON response
  5. Validates output with Pydantic
  6. Returns enhanced product listing


In [44]:
print("\n" + "="*50)
print("CREATING COMPLETE WORKFLOW FUNCTION")
print("="*50)

def process_product_listing(json_file_path):
    """
    Complete workflow: Load JSON → Validate → Enhance → Validate output
    
    This is the main function that ties everything together!
    
    Args:
        json_file_path: Path to JSON file with product data
        
    Returns:
        EnhancedProductListing object if successful, None if any step fails
    """
    
    print("\n" + "="*70)
    print("COMPLETE PRODUCT LISTING WORKFLOW")
    print("="*70)
    print(f"Processing: {json_file_path}")
    print("="*70)
    
    # STEP 1: Load and validate INPUT
    print("\n STEP 1: LOAD & VALIDATE INPUT")
    print("-" * 70)
    product = validate_product_json(json_file_path)
    
    if product is None:
        print("\n WORKFLOW FAILED: Input validation failed")
        print("   Fix the input data and try again!")
        return None
    
    print("\n Step 1 complete: Input validated successfully")
    
    # STEP 2: Enhance with ChatGPT
    print("\n STEP 2: ENHANCE WITH CHATGPT")
    print("-" * 70)
    enhanced_product = enhance_product_with_chatgpt(product)
    
    if enhanced_product is None:
        print("\n WORKFLOW FAILED: Enhancement failed")
        print("   Check API connection and try again!")
        return None
    
    print("\n Step 2 complete: Product enhanced successfully")
    
    # STEP 3: Success!
    print("\n" + "="*70)
    print(" WORKFLOW COMPLETE!")
    print("="*70)
    print("✓ Input validated")
    print("✓ Enhanced with ChatGPT")
    print("✓ Output validated")
    print("\n Ready to return enhanced listing to client!")
    
    return enhanced_product

print("\n✓ Complete workflow function created!")
print("\nThis function orchestrates the entire process:")
print("  1. Loads JSON file")
print("  2. Validates input data")
print("  3. Calls ChatGPT for enhancement")
print("  4. Validates output data")
print("  5. Returns final enhanced product")


CREATING COMPLETE WORKFLOW FUNCTION

✓ Complete workflow function created!

This function orchestrates the entire process:
  1. Loads JSON file
  2. Validates input data
  3. Calls ChatGPT for enhancement
  4. Validates output data
  5. Returns final enhanced product


In [47]:
print("\n" + "="*70)
print("TESTING COMPLETE WORKFLOW")
print("="*70)

# Test with a valid product
result = process_product_listing("test_data/valid_product.json")

if result:
    print("\n" + "="*70)
    print("FINAL RESULT - ENHANCED PRODUCT LISTING")
    print("="*70)
    print(f"\nProduct: {result.name}")
    print(f"Price: ${result.price}")
    print(f"Category: {result.category}")
    print(f"\n Marketing Description:")
    print(f"{result.marketing_description}")
    print(f"\n Key Benefits:")
    for i, benefit in enumerate(result.key_benefits, 1):
        print(f"  {i}. {benefit}")
    print(f"\n Target Audience:")
    print(f"{result.target_audience}")
    print(f"\n SEO Keywords:")
    print(f"{', '.join(result.seo_keywords)}")
    print("\n" + "="*70)
    print(" SUCCESS! Complete workflow executed perfectly!")
    print("="*70)
else:
    print("\n Workflow failed. Check errors above.")




TESTING COMPLETE WORKFLOW

COMPLETE PRODUCT LISTING WORKFLOW
Processing: test_data/valid_product.json

 STEP 1: LOAD & VALIDATE INPUT
----------------------------------------------------------------------

 Loading: test_data/valid_product.json
------------------------------------------------------------
 JSON file loaded successfully
   Data preview:
{
  "name": "Premium Wireless Headphones",
  "price": 149.99,
  "category": "Electronics",
  "description": "High-quality wireless headphones with active noise cancellation, 30-hour battery life, and premium sound quality. Perfect for music lovers and frequent travelers.",
  "brand": "AudioTech Pro"
  ... (truncated)

 VALIDATION PASSED!
   Product Name: Premium Wireless Headphones
   Price: $149.99
   Category: Electronics
   Description: High-quality wireless headphones with active noise...
   Brand: AudioTech Pro
   Features: 4 items

 Step 1 complete: Input validated successfully

 STEP 2: ENHANCE WITH CHATGPT
-----------------------

In [48]:
print("\n" + "="*70)
print("TESTING WITH INVALID INPUT")
print("="*70)
print("This should fail at Step 1 (input validation)")
print("It should NOT call ChatGPT API (saves money!)")
print("="*70)

# Test with invalid product (negative price)
result = process_product_listing("test_data/invalid_product_price.json")

if result is None:
    print("\n Correct behavior: Invalid input rejected before API call!")
    print("   This saves money by not calling ChatGPT on bad data.")
else:
    print("\n Unexpected: Invalid input should have been rejected!")


TESTING WITH INVALID INPUT
This should fail at Step 1 (input validation)
It should NOT call ChatGPT API (saves money!)

COMPLETE PRODUCT LISTING WORKFLOW
Processing: test_data/invalid_product_price.json

 STEP 1: LOAD & VALIDATE INPUT
----------------------------------------------------------------------

 Loading: test_data/invalid_product_price.json
------------------------------------------------------------
 JSON file loaded successfully
   Data preview:
{
  "name": "Broken Product",
  "price": -25.0,
  "category": "Electronics",
  "description": "This product has an invalid negative price and should be rejected by validation."
}
 VALIDATION FAILED!
   Reason: 1 validation error for ProductListing
price
  Input should be greater than 0 [type=greater_than, input_value=-25.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than

   Detailed validation errors:
     • Field: 'price'
       Error: Input should be greater than 0
       Type